In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [3]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [5]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
tx_augmented = build_poly_all_features(tx,2)
y,tx = build_model_data(tx_augmented,y)
y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [6]:
num_samples, num_features
tx.shape

(250000, 47)

In [7]:
from compute_gradient import *
from cost import *
from implementations import *

In [8]:
#Initial weight


#Maximum iterations through the whole set
max_iter = 100

#0.001 nan
#0.0005 nan

#Step-size
gammas = np.logspace(-6,-3, 7)
lambdas = np.logspace (-5,0,7)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 47), (250000, 1), array([[1.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]))

In [9]:
losses_tr = np.zeros((len(gammas),len(lambdas)))
losses_val = np.zeros((len(gammas),len(lambdas)))

k_fold = 2
seed = 1
loss_min = np.inf
initial_w = np.zeros((tx.shape[1],1))

#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    for ind_gamma ,gamma_ in enumerate (gammas):
        loss_training = 0
        loss_validation = 0
        for i,k in enumerate(range(k_fold)):
            
            tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
            w, loss_train_k = reg_logistic_regression(y_train, tx_train, lambda_, initial_w, max_iter, gamma_)
            loss_val_k = reg_logistic_loss (y_val,tx_val,w,lambda_)/len(y_val)
            loss_train_k /= len(y_train)
            loss_training += loss_train_k
            loss_validation += loss_val_k
        
        
        loss_training /= k_fold
        loss_validation /= k_fold
        print(loss_training)
        print(loss_validation)
        if (loss_validation < loss_min):
            weights_star = w
            lambda_star = lambda_
            gamma_star = gamma_
            loss_min = loss_validation
        
        losses_tr[ind_lambda][ind_gamma] = loss_training
        losses_val[ind_lambda][ind_gamma] = loss_validation



0.693038900853815
0.693038626049674
0.6926698493894505
0.6926687692757123
0.691516208195458
0.6915115249552872
0.6879823551204608
0.6879650486461031
0.6778106799235489
0.677749738877403
0.6531265553918006
0.6529375662217655
0.613318196161167
0.6126913813140342
0.6082108341655306
0.6082106929122877
0.6081276245536795
0.6081271735898788
0.6078662653520552
0.6078647975095669
0.6070571411113745
0.6070520866097321
0.6046614904898112
0.6046415394000191
0.5984393561679111
0.5983416034460913
inf
0.5866080968985525
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf
inf


In [11]:
lambdas

array([1.00000000e-05, 6.81292069e-05, 4.64158883e-04, 3.16227766e-03,
       2.15443469e-02, 1.46779927e-01, 1.00000000e+00])

In [12]:
gammas

array([1.00000000e-06, 3.16227766e-06, 1.00000000e-05, 3.16227766e-05,
       1.00000000e-04, 3.16227766e-04, 1.00000000e-03])

In [13]:
losses_tr, losses_val

(array([[0.6930389 , 0.69266985, 0.69151621, 0.68798236, 0.67781068,
         0.65312656, 0.6133182 ],
        [0.60821083, 0.60812762, 0.60786627, 0.60705714, 0.60466149,
         0.59843936,        inf],
        [       inf,        inf,        inf,        inf,        inf,
                inf,        inf],
        [       inf,        inf,        inf,        inf,        inf,
                inf,        inf],
        [       inf,        inf,        inf,        inf,        inf,
                inf,        inf],
        [       inf,        inf,        inf,        inf,        inf,
                inf,        inf],
        [       inf,        inf,        inf,        inf,        inf,
                inf,        inf]]),
 array([[0.69303863, 0.69266877, 0.69151152, 0.68796505, 0.67774974,
         0.65293757, 0.61269138],
        [0.60821069, 0.60812717, 0.6078648 , 0.60705209, 0.60464154,
         0.5983416 , 0.5866081 ],
        [       inf,        inf,        inf,        inf,        inf,
  

In [15]:
loss_min, lambda_star, gamma_star

(0.5866080968985525, 6.812920690579608e-05, 0.001)

In [16]:
#PREDICT TRAINING ACCURACY
y_pred_training = predict_labels(weights_star, tx,'logistic')
count = 0
for i in range(num_samples):
    if (y_pred_training[i] == -1 and y[i] == 0):
        count += 1
    elif (y_pred_training[i] == y[i]):
        count += 1
accuracy = (count *100) / num_samples

accuracy

67.5524

In [198]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [203]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [206]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [207]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [210]:
tx_test_augmented = build_poly_all_features(tx_test_clean,2)

(568238, 23)

In [213]:
tx_test_augmented.shape

(568238, 46)

In [214]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [215]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre2.csv'
y_pred = predict_labels(weights_star, tx_test,'logistic')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)